# CASSINI TO UTM CONVERSION

In [29]:
import pandas as pd
import numpy as np
import utm

In [30]:
sys_params = pd.read_csv("./data/parameters.csv")
cood_controls = pd.read_csv("./data/controls.csv")

In [31]:
cassini_contols = cood_controls.iloc[:,1:3]
cassini_contols

,cass_northings,cass_eastings
0,-54102.1,-182848.4
1,-54093.1,-128079.9
2,-54089.0,-91567.5
3,-144786.5,-182828.2
4,-144775.6,-128066.6
5,-144770.8,-91559.2
6,-90366.3,-128075.0


In [32]:
def convert_to_conformal(df):
    e = df['cass_eastings']
    a = float(sys_params.iloc[2,1])
    b = float(sys_params.iloc[3,1])
    conformal_easting = e + ((e**3)/(6*a*b)) + ((e**5)/(24*(a**2)*(b**2)))
    return conformal_easting

In [33]:
cassini_contols['conformal_easting'] = cassini_contols.apply(convert_to_conformal,axis=1)
cassini_contols

,cass_northings,cass_eastings,conformal_easting
0,-54102.1,-182848.4,-182850.734652
1,-54093.1,-128079.9,-128080.702393
2,-54089.0,-91567.5,-91567.793202
3,-144786.5,-182828.2,-182830.533878
4,-144775.6,-128066.6,-128067.402143
5,-144770.8,-91559.2,-91559.493122
6,-90366.3,-128075.0,-128075.802301


## NUMPY MATRIX FORMULATION

In [34]:
utm_northings = cood_controls.iloc[:,3]
utm_eastings = cood_controls.iloc[:,4]
i_matrix = pd.concat([utm_eastings, utm_northings], ignore_index=True)
i_matrix.to_numpy()

array([ 221784.7,  238487.3,  249621.4,  221813.7,  238514.6,  249647.4,
        238497.6, 9861717.5, 9861732.9, 9861742.7, 9834060.9, 9834079.4,
       9834091.1, 9850671.5])

In [55]:
i_matrix.shape

(14,)

In [35]:
cass_simple = cassini_contols[['conformal_easting','cass_northings']]
cass_simple.columns = ['x','y']
cass_simple['xx'] = cass_simple['x']**2
cass_simple['xy'] = cass_simple['x']*cass_simple['y']
cass_simple['yy'] = cass_simple['y']**2
cass_simple.to_numpy()
cass_simple

,x,y,xx,xy,yy
0,-182850.734652,-54102.1,3.343439e+10,9.892609e+09,2.927037e+09
1,-128080.702393,-54093.1,1.640467e+10,6.928282e+09,2.926063e+09
2,-91567.793202,-54089.0,8.384661e+09,4.952810e+09,2.925620e+09
3,-182830.533878,-144786.5,3.342700e+10,2.647139e+10,2.096313e+10
4,-128067.402143,-144775.6,1.640126e+10,1.854103e+10,2.095997e+10
5,-91559.493122,-144770.8,8.383141e+09,1.325514e+10,2.095858e+10
6,-128075.802301,-90366.3,1.640341e+10,1.157374e+10,8.166068e+09


In [36]:
type(cass_simple)

pandas.core.frame.DataFrame

In [37]:
# add 7,5 array of zeros to the bottom of the cass_simple array
zeros = np.zeros((7,5))
cass_simple1 = np.vstack((cass_simple,zeros))
cass_simple1

array([[-1.82850735e+05, -5.41021000e+04,  3.34343912e+10,
         9.89260873e+09,  2.92703722e+09],
       [-1.28080702e+05, -5.40931000e+04,  1.64046663e+10,
         6.92828224e+09,  2.92606347e+09],
       [-9.15677932e+04, -5.40890000e+04,  8.38466075e+09,
         4.95281037e+09,  2.92561992e+09],
       [-1.82830534e+05, -1.44786500e+05,  3.34270041e+10,
         2.64713931e+10,  2.09631306e+10],
       [-1.28067402e+05, -1.44775600e+05,  1.64012595e+10,
         1.85410350e+10,  2.09599744e+10],
       [-9.15594931e+04, -1.44770800e+05,  8.38314078e+09,
         1.32551411e+10,  2.09585845e+10],
       [-1.28075802e+05, -9.03663000e+04,  1.64034111e+10,
         1.15737364e+10,  8.16606818e+09],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00

In [38]:
# stack the zeros 7,5 array top of the cass_simple array
zeros = np.zeros((7,5))
cass_simple2 = np.vstack((zeros,cass_simple))

In [39]:
# horizontal stack the cass_simple1 and cass_simple2 arrays
cass_simple3 = np.hstack((cass_simple1,cass_simple2))
cass_simple3

array([[-1.82850735e+05, -5.41021000e+04,  3.34343912e+10,
         9.89260873e+09,  2.92703722e+09,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-1.28080702e+05, -5.40931000e+04,  1.64046663e+10,
         6.92828224e+09,  2.92606347e+09,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-9.15677932e+04, -5.40890000e+04,  8.38466075e+09,
         4.95281037e+09,  2.92561992e+09,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-1.82830534e+05, -1.44786500e+05,  3.34270041e+10,
         2.64713931e+10,  2.09631306e+10,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-1.28067402e+05, -1.44775600e+05,  1.64012595e+10,
         1.85410350e+10,  2.09599744e+10,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.

In [40]:
# create a 7,2 array of ones and zeros (1,0)
ones = np.ones((7,1))
zeros = np.zeros((7,1))
ones_zeros = np.hstack((ones,zeros))
zeros_ones = np.hstack((zeros,ones))
# vertical stack the ones_zeros and zeros_ones arrays
one_zero = np.vstack((ones_zeros,zeros_ones))
one_zero

# horizontal stack the cass_simple3 and one_zero arrays
cass_simple_final = np.hstack((cass_simple3,one_zero))
cass_simple_final
a_matrix = cass_simple_final

In [41]:
a_matrix

array([[-1.82850735e+05, -5.41021000e+04,  3.34343912e+10,
         9.89260873e+09,  2.92703722e+09,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-1.28080702e+05, -5.40931000e+04,  1.64046663e+10,
         6.92828224e+09,  2.92606347e+09,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-9.15677932e+04, -5.40890000e+04,  8.38466075e+09,
         4.95281037e+09,  2.92561992e+09,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-1.82830534e+05, -1.44786500e+05,  3.34270041e+10,
         2.64713931e+10,  2.09631306e+10,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-1.28067402e+05, -1.44775600e+05,  1.6401259

## Matrix Calculation

In [42]:
a_matrix_transpose = a_matrix.transpose()
a_matrix_transpose

array([[-1.82850735e+05, -1.28080702e+05, -9.15677932e+04,
        -1.82830534e+05, -1.28067402e+05, -9.15594931e+04,
        -1.28075802e+05,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-5.41021000e+04, -5.40931000e+04, -5.40890000e+04,
        -1.44786500e+05, -1.44775600e+05, -1.44770800e+05,
        -9.03663000e+04,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 3.34343912e+10,  1.64046663e+10,  8.38466075e+09,
         3.34270041e+10,  1.64012595e+10,  8.38314078e+09,
         1.64034111e+10,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 9.89260873e+09,  6.92828224e+09,  4.95281037e+09,
         2.64713931e+10,  1.85410350e+10,  1.32551411e+10,
         1.15737364e+10,  0.00000000e+00,  0.

In [43]:
a_matrix_transpose_a_matrix = np.matmul(a_matrix_transpose,a_matrix)
a_matrix_transpose_a_matrix

array([[ 1.32838534e+11,  9.16150069e+10, -2.00627689e+16,
        -1.30599986e+16, -1.06596987e+16,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -9.33032462e+05,  0.00000000e+00],
       [ 9.16150069e+10,  7.98264783e+10, -1.30599986e+16,
        -1.06596987e+16, -1.03166821e+16,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -6.86983400e+05,  0.00000000e+00],
       [-2.00627689e+16, -1.30599986e+16,  3.18298899e+21,
         1.97586130e+21,  1.52454990e+21,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.32838534e+11,  0.00000000e+00],
       [-1.30599986e+16, -1.06596987e+16,  1.97586130e+21,
         1.52454990e+21,  1.37958247e+21,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  9.16150069e+10,  0.00000000e+00],
       [-1.06596987e+16, -1.03166821e+16,  1.5245499

In [44]:
a_matrix_transpose_i_matrix = np.matmul(a_matrix_transpose,i_matrix)
a_matrix_transpose_i_matrix

array([-2.18460020e+11, -1.62741830e+11,  3.07520425e+16,  2.14461091e+16,
        1.89063116e+16, -9.18876664e+12, -6.76183511e+12,  1.30822599e+18,
        9.01742965e+17,  7.85398021e+17,  1.65836670e+06,  6.89380960e+07])

In [45]:
# inverse of a_matrix_transpose_a_matrix

a_t_a_inv = np.linalg.inv(a_matrix_transpose_a_matrix)
a_t_a_inv

array([[ 1.53084637e-08, -6.11399582e-09,  5.27177632e-14,
         5.75400606e-15, -3.46243326e-14, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  7.59559263e-04, -0.00000000e+00],
       [-6.11399582e-09,  1.66447026e-08, -2.51869841e-14,
         8.01868538e-15,  7.75055041e-14, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  3.07750928e-04, -0.00000000e+00],
       [ 5.27177632e-14, -2.51869841e-14,  1.90087603e-19,
         9.19754410e-23, -1.26708748e-19,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  2.39138180e-09,  0.00000000e+00],
       [ 5.75400606e-15,  8.01868538e-15,  9.19754410e-23,
         5.76133422e-20,  1.45509850e-21, -0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  7.81536882e-10, -0.00000000e+00],
       [-3.46243326e-14,  7.75055041e-14, -1.2670874

In [46]:
# multiply a_t_a_inv and a_matrix_transpose_i_matrix
sol_parameters = np.matmul(a_t_a_inv,a_matrix_transpose_i_matrix)
sol_parameters

array([ 3.04875211e-01, -2.16197709e-04, -2.56154367e-10, -7.66805308e-11,
        2.50167219e-10,  2.14365165e-04,  3.04877265e-01,  3.48804319e-11,
       -5.08559105e-10, -3.11577431e-11,  2.77528247e+05,  9.87825515e+06])

In [47]:
# convert the sol_parameters array to a dictionary with keys a1,a2,a3,a4,a5,b1,b2,b3,b4,b5,a0,b0
sol_parameters_dict = {
    'a1':sol_parameters[0],
    'a2':sol_parameters[1],
    'a3':sol_parameters[2],
    'a4':sol_parameters[3],
    'a5':sol_parameters[4],
    'b1':sol_parameters[5],
    'b2':sol_parameters[6],
    'b3':sol_parameters[7],
    'b4':sol_parameters[8],
    'b5':sol_parameters[9],
    'a0':sol_parameters[10],
    'b0':sol_parameters[11]
    }

In [48]:
sol_parameters_dict['a0']

277528.2466442585

## SOLUTION FUNCTION

In [49]:
cassini_contols

,cass_northings,cass_eastings,conformal_easting
0,-54102.1,-182848.4,-182850.734652
1,-54093.1,-128079.9,-128080.702393
2,-54089.0,-91567.5,-91567.793202
3,-144786.5,-182828.2,-182830.533878
4,-144775.6,-128066.6,-128067.402143
5,-144770.8,-91559.2,-91559.493122
6,-90366.3,-128075.0,-128075.802301


In [50]:
def northing_comp(df):
    y = df['cass_northings']
    x = df['conformal_easting']
    b0 = sol_parameters_dict['b0']
    b1 = sol_parameters_dict['b1']
    b2 = sol_parameters_dict['b2']
    b3 = sol_parameters_dict['b3']
    b4 = sol_parameters_dict['b4']
    b5 = sol_parameters_dict['b5']

    northing = b0+b1*x+b2*y+b3*x**2+b4*x*y+b5*y**2
    return np.round(northing,4)

def easting_comp(df):
    y = df['cass_northings']
    x = df['conformal_easting']
    a0 = sol_parameters_dict['a0']
    a1 = sol_parameters_dict['a1']
    a2 = sol_parameters_dict['a2']
    a3 = sol_parameters_dict['a3']
    a4 = sol_parameters_dict['a4']
    a5 = sol_parameters_dict['a5']

    easting = a0+a1*x+a2*y+a3*x**2+a4*x*y+a5*y**2
    return np.round(easting,4)


In [51]:
def convert_to_latlon(df):
    df_np = df.to_numpy()
    np_latlon = utm.to_latlon(df['easting'],df['northing'], 37, 'M')
    df_latlon = pd.DataFrame(list(np_latlon)).transpose()
    df_latlon.columns = ['latitude','longitude']
    # paste the original dataframe to the latlon dataframe
    df_latlon = pd.concat([cood_controls["stn"],df,df_latlon],axis=1)
    return df_latlon

In [52]:
converted_utm = pd.DataFrame(
    {
        'northing':cassini_contols.apply(northing_comp,axis=1),
        'easting':cassini_contols.apply(easting_comp,axis=1),
    }
)

In [53]:
convert_to_latlon(converted_utm)

,stn,northing,easting,latitude,longitude
0,1,-138282.4995,221784.6964,-1.249885,36.499957
1,2,-138267.1013,238487.3089,-1.249885,36.649959
2,3,-138257.2993,249621.3946,-1.249885,36.749961
3,4,-165939.1005,221813.7036,-1.499862,36.499957
4,5,-165920.5987,238514.5911,-1.499862,36.649959
5,6,-165908.9008,249647.4054,-1.499862,36.749960
6,7,-149328.5000,238497.6000,-1.349876,36.649959


In [54]:
converted_utm.to_csv("./data/converted_utm.csv")